In [2]:
import pandas as pd
import numpy as np
import json
import gzip
import os
from os import listdir
import seaborn as sns
import matplotlib.pyplot as plt
import py_entitymatching as em
import py_stringmatching as sm
from scipy.io import arff
from sklearn import preprocessing


In [3]:
#read table to dataframe and create train/test split with equal distribution of labels within table
#example small book
small = pd.read_csv('../../src/data/schemafiltereddata/small_tables.csv')
small_books = small[small['Class']=='Book']
small_books

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,...,Restaurant_menu,Restaurant_name,Restaurant_openinghours,Restaurant_openinghoursspecification,Restaurant_pricerange,Restaurant_servescuisine,Restaurant_telephone,TVEpisode_episodenumber,TVEpisode_name,TVEpisode_partofseries
0,Book_aardvark-books.com_September2020.json.gz,Book,26.0,9.0,7.0,5,0.027473,0.027473,35.0,0,...,0,0,0,0,0,0,0,0,0,0
1,Book_americanreadingathome.com_September2020.json.gz,Book,33.0,9.0,5.0,3,0.018182,0.018182,15.0,0,...,0,0,0,0,0,0,0,0,0,0
2,Book_barmatrioshka.com_September2020.json.gz,Book,35.0,11.0,9.0,0,0.000000,0.000000,0.0,1,...,0,0,0,0,0,0,0,0,0,0
3,Book_bibliobeat.com_September2020.json.gz,Book,22.0,10.0,6.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,Book_bookapy.com_September2020.json.gz,Book,26.0,13.0,8.0,11,0.052885,0.052885,88.0,0,...,0,0,0,0,0,0,0,0,0,0
5,Book_books.org_September2020.json.gz,Book,36.0,8.0,5.0,11,0.061111,0.061111,55.0,0,...,0,0,0,0,0,0,0,0,0,0
6,Book_capitalsoftworks.co.uk_September2020.json.gz,Book,25.0,13.0,7.0,17,0.097143,0.097143,119.0,0,...,0,0,0,0,0,0,0,0,0,0
7,Book_carlosruizzafon.co.uk_September2020.json.gz,Book,27.0,10.0,5.0,1,0.007407,0.007407,5.0,0,...,0,0,0,0,0,0,0,0,0,0
8,Book_clairenorth.com_September2020.json.gz,Book,22.0,11.0,6.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,0,0
9,Book_cressidacowell.co.uk_September2020.json.gz,Book,21.0,11.0,6.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = small_books
train_size = int(len(X) * 0.7)
train, test = X[0:train_size], X[train_size:len(X)]
#X_train, y_train, X_test, y_test = iterative_train_test_split(train,test, test_size = 0.2)
#test = test.values
print('Total Counts: %d' % (len(X)))
print('Train Counts: %d' % (len(train)))
print('Testing Counts: %d' % (len(test)))

In [36]:
from skmultilearn.model_selection import iterative_train_test_split

In [3]:
path='/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/schemafiltereddata/Book/'

In [4]:
test_books = pd.read_json(path+'Book_aardvark-books.com_September2020.json.gz', compression='gzip', lines=True)
test_books

,row_id,name,author,publisher,isbn,bookformat,datepublished,offers,page_url
0,0,Pacific Steam The British Pacific Locomotive,"{'name': 'Evans, Martin'}",{'name': 'Model Aeronautical Press'},None,Hardcover,1960-01-01,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
1,1,Early Aviation In North Wales,"{'name': 'Sloan, Roy'}",{'name': 'Gwasg Carreg Gwalch'},0863811191,Paperback,1989-05-01,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
2,2,Cessna 150 Pilots Guide: A Pilot's Guide (The ...,"{'name': 'Pratt, Jeremy M.'}",{'name': 'Airplan Flight Equipment Ltd'},187478325X,Paperback,1993-07-01,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
3,3,Early MG,{'name': 'Jennings P L'},{'name': 'P. L. Jennings'},0951450905,Hardcover,1989-01-01,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
4,4,"Selected Poems, 1908-59","{'name': 'Pound, Ezra'}",{'name': 'Faber & Faber'},None,Hardcover,1975-10-27,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/rarebooks/...
5,5,Steaming Through Britain: Complete Guide to Br...,{'name': 'Chris Ellis'},{'name': 'Conway Maritime Press Ltd'},184486121X,Hardcover,2010-10-12,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
6,6,Cruising: A Manual for Small Cruiser Sailing,"{'name': 'Sleightholme, J.D.'}",{'name': 'Adlard Coles Nautical'},0713634774,Paperback,1990-09-03,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
7,7,The Trains We Loved,"{'name': 'Whitehouse, Patrick, St.John Thomas,...",{'name': 'David & Charles'},None,Hardcover,1994-09-30,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
8,8,Steam Portfolios: North Eastern Steam Bk. 5,"{'name': 'Johnson, M.'}",{'name': 'Ian Allan Publishing'},0711018693,Hardcover,1989-11-01,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
9,9,GREAT WESTERN: ALBUM NO.2.,"{'name': 'Riley, R.C.'}",{'name': 'Ian Allen'},0711000735,Hardcover,1971-01-01,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...


In [33]:
em.is_dfinfo_present(test)

True

In [31]:
test_books.to_csv('test_csv.csv')
test = em.read_csv_metadata('test_csv.csv')
test['ID'] = range(0, len(test))
em.set_key(test, 'ID')

Metadata file is not present in the given path; proceeding to read the csv file.


True

In [ ]:
train_test = em.split_train_test(test, train_proportion=0.5)
train, test = train_test['train'], train_test['test']

In [ ]:
X_train, y_train, X_test, y_test = iterative_train_test_split(test, test_size = 0.2)

In [51]:
from skmultilearn.dataset import load_dataset
X,y, _, _ = load_dataset('test')

ModuleNotFoundError: No module named 'arff'

In [68]:
X = test_books
train_size = int(len(X) * 0.7)
train, test = X[0:train_size], X[train_size:len(X)]
#X_train, y_train, X_test, y_test = iterative_train_test_split(train,test, test_size = 0.2)
#test = test.values
print('Total Counts: %d' % (len(X)))
print('Train Counts: %d' % (len(train)))
print('Testing Counts: %d' % (len(test)))

Total Counts: 26
Train Counts: 18
Testing Counts: 8


In [59]:
train = pd.DataFrame(train)
train

,row_id,name,author,publisher,isbn,bookformat,datepublished,offers,page_url
0,0,Pacific Steam The British Pacific Locomotive,"{'name': 'Evans, Martin'}",{'name': 'Model Aeronautical Press'},None,Hardcover,1960-01-01,"{'pricevaliduntil': '2020-10-11', 'pricecurrency': 'GBP', 'itemcondition': 'http://schema.org/Us...",https://www.aardvark-books.com/shop/other-non-fiction/transport/railways/375848
1,1,Early Aviation In North Wales,"{'name': 'Sloan, Roy'}",{'name': 'Gwasg Carreg Gwalch'},0863811191,Paperback,1989-05-01,"{'pricevaliduntil': '2020-10-11', 'pricecurrency': 'GBP', 'itemcondition': 'http://schema.org/Us...",https://www.aardvark-books.com/shop/other-non-fiction/transport/438077
2,2,Cessna 150 Pilots Guide: A Pilot's Guide (The pilot's guide series),"{'name': 'Pratt, Jeremy M.'}",{'name': 'Airplan Flight Equipment Ltd'},187478325X,Paperback,1993-07-01,"{'pricevaliduntil': '2020-10-11', 'pricecurrency': 'GBP', 'price': '4.19', 'itemcondition': 'htt...",https://www.aardvark-books.com/shop/other-non-fiction/transport/438141
3,3,Early MG,{'name': 'Jennings P L'},{'name': 'P. L. Jennings'},0951450905,Hardcover,1989-01-01,"{'pricevaliduntil': '2020-10-11', 'pricecurrency': 'GBP', 'price': '45.00', 'itemcondition': 'ht...",https://www.aardvark-books.com/shop/other-non-fiction/transport/431248
4,4,"Selected Poems, 1908-59","{'name': 'Pound, Ezra'}",{'name': 'Faber & Faber'},None,Hardcover,1975-10-27,"{'pricevaliduntil': '2020-10-11', 'pricecurrency': 'GBP', 'itemcondition': 'http://schema.org/Us...",https://www.aardvark-books.com/shop/rarebooks/modern-first-editions/383487
5,5,"Steaming Through Britain: Complete Guide to Britain's Historic Railways, Heritage Lines and Trains",{'name': 'Chris Ellis'},{'name': 'Conway Maritime Press Ltd'},184486121X,Hardcover,2010-10-12,"{'pricevaliduntil': '2020-10-11', 'pricecurrency': 'GBP', 'price': '3.96', 'itemcondition': 'htt...",https://www.aardvark-books.com/shop/other-non-fiction/transport/railways/434032
6,6,Cruising: A Manual for Small Cruiser Sailing,"{'name': 'Sleightholme, J.D.'}",{'name': 'Adlard Coles Nautical'},0713634774,Paperback,1990-09-03,"{'pricevaliduntil': '2020-10-11', 'pricecurrency': 'GBP', 'price': '0.01', 'itemcondition': 'htt...",https://www.aardvark-books.com/shop/other-non-fiction/transport/413691
7,7,The Trains We Loved,"{'name': 'Whitehouse, Patrick, St.John Thomas, David'}",{'name': 'David & Charles'},None,Hardcover,1994-09-30,"{'pricevaliduntil': '2020-10-11', 'pricecurrency': 'GBP', 'itemcondition': 'http://schema.org/Us...",https://www.aardvark-books.com/shop/other-non-fiction/transport/railways/367024
8,8,Steam Portfolios: North Eastern Steam Bk. 5,"{'name': 'Johnson, M.'}",{'name': 'Ian Allan Publishing'},0711018693,Hardcover,1989-11-01,"{'pricevaliduntil': '2020-10-11', 'pricecurrency': 'GBP', 'sku': 'mon0000260651', 'itemcondition...",https://www.aardvark-books.com/shop/other-non-fiction/transport/railways/424555
9,9,GREAT WESTERN: ALBUM NO.2.,"{'name': 'Riley, R.C.'}",{'name': 'Ian Allen'},0711000735,Hardcover,1971-01-01,"{'pricevaliduntil': '2020-10-11', 'pricecurrency': 'GBP', 'itemcondition': 'http://schema.org/Us...",https://www.aardvark-books.com/shop/other-non-fiction/transport/railways/412543


In [65]:
test = pd.DataFrame(test)
test

,0,1,2,3,4,5,6,7,8
0,18,Restless Spirit,"{'name': 'Robertson, Don'}",{'name': 'Cross Publishing'},1873295405,Hardcover,1994-07-01,"{'pricevaliduntil': '2020-10-11', 'pricecurrency': 'GBP', 'itemcondition': 'http://schema.org/Us...",https://www.aardvark-books.com/shop/other-non-fiction/transport/375758
1,19,The Official DSA Theory Test for Motorcyclists 2012,{'name': 'Driving Standards Agency (Great Britain)'},{'name': 'Stationery Office Books'},0115531866,Paperback,2011-09-19,"{'pricevaliduntil': '2020-10-11', 'itemcondition': 'http://schema.org/NewCondition', 'sku': 'mon...",https://www.aardvark-books.com/shop/other-non-fiction/transport/383261
2,20,Metro Memories,"{'name': 'Ron Pigram, Dennis Edwards'}",{'name': 'Godfrey Cave Associates Ltd'},1870630955,Hardcover,1988-12-31,"{'pricevaliduntil': '2020-10-11', 'pricecurrency': 'GBP', 'itemcondition': 'http://schema.org/Us...",https://www.aardvark-books.com/shop/other-non-fiction/transport/railways/369831
3,21,Drive Style: Horst A. Friedrichs,{'name': 'Horst A. Friedrichs'},{'name': 'Prestel'},3791348590,Hardcover,2013-08-30,"{'pricevaliduntil': '2020-10-11', 'itemcondition': 'http://schema.org/NewCondition', 'pricecurre...",https://www.aardvark-books.com/shop/other-non-fiction/transport/432250
4,22,Along Lost Lines P/b,"{'name': 'Atterbury, Paul'}",{'name': 'David & Charles'},0715327062,Paperback,2009-11-09,"{'pricevaliduntil': '2020-10-11', 'sku': 'mon0000182620', 'itemcondition': 'http://schema.org/Ne...",https://www.aardvark-books.com/shop/other-non-fiction/transport/railways/367022
5,23,"Roads and Rails of Birmingham, 1900-39","{'name': 'Coxon, R.T.'}",{'name': 'Littlehampton Book Services Ltd'},0711009139,Hardcover,1979-05-01,"{'pricevaliduntil': '2020-10-11', 'pricecurrency': 'GBP', 'itemcondition': 'http://schema.org/Us...",https://www.aardvark-books.com/shop/other-non-fiction/transport/434653
6,24,Voyages: To the New World and Beyond,None,None,1553655737,Hardcover,0000-00-00,"{'pricevaliduntil': '2020-10-11', 'itemcondition': 'http://schema.org/NewCondition', 'pricecurre...",https://www.aardvark-books.com/shop/other-non-fiction/transport/432508
7,25,Classic Ships: Romance and Reality (A Channel Four book),"{'name': 'Faith, Nicholas'}",{'name': 'Boxtree Ltd'},075221022X,Hardcover,1996-01-29,"{'pricevaliduntil': '2020-10-11', 'pricecurrency': 'GBP', 'price': '3.96', 'sku': 'mon0000189089...",https://www.aardvark-books.com/shop/other-non-fiction/transport/368332
